In [12]:
print("Test!")

Test!


In [13]:
# TODO: Auth file somehow?
import ee
import numpy as np
import geopandas
# import matplotlib.pyplot as plt

# from bqplot import pyplot as plt
# from ipyleaflet import *

# ee.Authenticate();
# ee.Initialize();

import geemap
import pandas as pd


In [24]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map.set_options(mapTypeId='SATELLITE');

In [15]:
shp_path = './Lhasa/Lhasa_RC_DGO2km_updated.shp';
lhasa_shp = geemap.shp_to_ee(shp_path);

KeyboardInterrupt: 

In [ ]:
def getCover(image, AOI, scale):
    totPixels = ee.Number(image.unmask(1).reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    actPixels = ee.Number(image.reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    percCover = actPixels.divide(totPixels).multiply(100).round();
    return image.set('percCover', percCover);

def maskL8sr(image):
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    qa = image.select('QA_PIXEL');
    mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)));
    return image.updateMask(mask);

def addMNDWI(image):
    return image.addBands(srcImg=image.normalizedDifference(['B2','B5']).rename('MNDWI')); # B2=greenl B4=SWIR

def addIS_WATER_MNDWI(image):
    return image.addBands(image.expression('MNDWI >  0.0', {
            'MNDWI': image.select('MNDWI'),
            }).rename('IS_WATER_MNDWI'));
def IS_WATER_INDIC(INDIC, ImColl):
    return ImColl.select('IS_WATER_' + INDIC).map(lambda image: image.updateMask(image.neq(0)));

def process_polygon(shp, dgo=1, START_DATE='1988-04-06', END_DATE='1988-04-08'):
    filter = ee.Filter.inList('DGO_FID', [dgo]);
    AOI = shp.filter(filter);
    filtered_DGO = AOI;
    AOI_ID=f'middle_ID{1}';

    START = ee.Date(START_DATE);
    END = ee.Date(END_DATE);
    # DAY_DIFF = ee.Date(END_DATE).difference(START, 'day');
    # DAY_RANGE = ee.List.sequence(0,DAY_DIFF.subtract(1),1).map(lambda day: START.advance(day,'day'));

    #  /* Working resolution, minimum 30 meters, read
    #                          https://developers.google.com/earth-engine/guides/scale
    #                          to learn how scales/images pyramids work in GEE */
    MIN_PIX = 2; # // To filter small water polygons

    eightCo = True;      # // eightConnected boolean for vectorization
    nCo = (eightCo+1)*4; #  // NConnected integer for file exports

    #/* int or float, multiplied by LANDSAT_SCALE, becomes the tolerance 
    #                                  of the simplification of the water polygons */

    # Visulization parameters :

    NDWI_MIN = -0.3;
    NDWI_MAX = 0.3;

    NDVI_MIN = -0.5;
    NDVI_MAX = 0.5;

    CLOUD_PAL = ['gray', 'white'];
    NDWI_PAL = ['black','white','blue'];
    NDVI_PAL = ['black','white','green'];

    LANDSAT_FULL = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','QA_PIXEL'])
        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL'])));
    LANDSAT_FULL = (LANDSAT_FULL
                        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                        .merge(ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))   
                    );

    LANDSAT = (LANDSAT_FULL
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER))
        .map(maskL8sr)
        .filterBounds(AOI));

    LANDSAT = LANDSAT.map(lambda image: getCover(image, AOI, LANDSAT_SCALE));
    LANDSAT = LANDSAT.filterMetadata('percCover', 'greater_than', 98);
    LANDSAT = LANDSAT.map(addMNDWI);

    LANDSAT = LANDSAT.map(addIS_WATER_MNDWI);

    IS_WATER_MNDWI = IS_WATER_INDIC('MNDWI', LANDSAT);
    return AOI, LANDSAT, IS_WATER_MNDWI;


CLOUD_FILTER = 80;
LANDSAT_SCALE = 30;
SIMPLIFY_TOLERANCE_N = 2;
AOI, LANDSAT, IS_WATER_MNDWI = process_polygon(shp=lhasa_shp, dgo=1);

Map.addLayer(lhasa_shp, {}, 'Lhasa River SHP');
Map.addLayer(IS_WATER_MNDWI, {'palette': 'Reds'}, 'water MNDWI', True);
Map.zoom = 11
Map.center_object(lhasa_shp);

In [ ]:
Map

Map(center=[29.651569731000073, 91.3219064102816], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [16]:
def VECTORIZE_FOR_AOI(AOI):
    def VECTORIZE(image):
        DAY_VECTORS = image.reduceToVectors(**{
            "geometry" : AOI,
            "scale" : LANDSAT_SCALE,
            "eightConnected" : True,
            "maxPixels" : 1e12,
            "geometryType" : 'polygon'
        });
        return DAY_VECTORS.set('DATE_ACQUIRED',image.get('DATE_ACQUIRED'));
    return VECTORIZE;

# Add area info to filter out isolated pixels
def ADDPROPS_FEAT(feature): 
    return feature.set({'AREA' : feature.area(LANDSAT_SCALE)});

def WRAP_COL_FOR_AOI(AOI):
    def WRAP_COL(collection):
        MIN_PIX=2
        collection = (ee.FeatureCollection(collection).map(ADDPROPS_FEAT)
            .filterMetadata('AREA', 'greater_than', MIN_PIX * LANDSAT_SCALE * LANDSAT_SCALE));
        time_start = collection.get('DATE_ACQUIRED');
        collection.map(lambda feature: feature.set({'DATE_ACQUIRED' : time_start}));
        geom = collection.geometry();
        geom_smooth = collection.geometry().simplify(**{'maxError' : SIMPLIFY_TOLERANCE_N*LANDSAT_SCALE});
        polygon = AOI.geometry(); #this is the polygon of the AOI/DGO
        polygon_smooth = AOI.geometry().simplify(**{'maxError' : SIMPLIFY_TOLERANCE_N*LANDSAT_SCALE})

        return ee.Feature(geom, {
            'TOTAL_PERIM' : geom.perimeter(LANDSAT_SCALE),
            'AREA' : geom.area(LANDSAT_SCALE),
            'TOTAL_PERIM_SMOOTH' : geom_smooth.perimeter(LANDSAT_SCALE),
            'AREA_SMOOTH' : geom_smooth.area(LANDSAT_SCALE),
            'AREA_POLYGON' : polygon_smooth.area(LANDSAT_SCALE), # this is the polygon of the AOI/DGO
            'SENSING_TIME' : collection.get('DATE_ACQUIRED')
        });
    return WRAP_COL;

def WATER_VEC(AOI, ImColl):
  WVEC = ImColl.map(VECTORIZE_FOR_AOI(AOI));
  return ee.FeatureCollection(WVEC.map(WRAP_COL_FOR_AOI(AOI)));


RIVER_PROPERTIES = [];
# TODO: Read DGO Count from shp 
for i in range(1, 78):
    # TODO: Print using ipython notify (progress bar...)
    print("DGO:", i, "/", 78);
    AOI, LANDSAT, IS_WATER_MNDWI = process_polygon(shp=lhasa_shp, dgo=1, START_DATE='1980-01-01', END_DATE='2100-01-01');
    ISWV_MNDWI = WATER_VEC(AOI=AOI, ImColl=IS_WATER_MNDWI);
    info = ISWV_MNDWI.getInfo(); # PROCESS AND PULL RESULT
    property_list = map(lambda x: {"ID": i, **x['properties']}, info['features']);
    # Map over properties (calculate Braiding Index e.t.c.)
    RIVER_PROPERTIES += list(property_list);

pd.DataFrame(RIVER_PROPERTIES).to_csv("test.csv");
print("Done! Saving as csv");


DGO: 1 / 78
DGO: 2 / 78
DGO: 3 / 78
DGO: 4 / 78
Done! Saving as csv


In [27]:
Map.addLayer(ISWV_MNDWI, {}, 'water shp')
Map.center_object(ISWV_MNDWI)
Map

Map(bottom=1851.0, center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [26]:
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [17]:
df = pd.read_csv("test.csv")
df

,Unnamed: 0,ID,AREA,AREA_POLYGON,AREA_SMOOTH,SENSING_TIME,TOTAL_PERIM,TOTAL_PERIM_SMOOTH
0,0,1,125306.491357,527573.551158,124405.009786,2013-04-02,5881.519719,4706.465915
1,1,1,141533.260363,527573.551158,131616.911104,2013-04-28,6181.598208,4798.390830
2,2,1,239795.287334,527573.551158,236189.348006,2013-06-15,6601.706987,5081.714263
3,3,1,305603.775169,527573.551158,294785.952139,2013-08-02,7081.830913,5554.341860
4,4,1,279460.683319,527573.551158,278108.456690,2013-08-18,6481.676044,5089.323290
...,...,...,...,...,...,...,...,...
1167,1167,4,200129.875614,527573.551158,196523.950792,2011-05-25,6421.660288,4972.174738
1168,1168,4,212750.698667,527573.551158,204186.552740,2011-06-10,6421.660484,4922.850387
1169,1169,4,265036.907988,527573.551158,261430.947612,2011-08-29,6421.660565,4993.622772
1170,1170,4,265036.894202,527573.551158,265938.356048,2011-09-14,6661.722153,5293.677147


In [18]:
df['BRAIDING_INDEX'] = (df['TOTAL_PERIM_SMOOTH']/2)/2000; # Calculate braiding index
df

,Unnamed: 0,ID,AREA,AREA_POLYGON,AREA_SMOOTH,SENSING_TIME,TOTAL_PERIM,TOTAL_PERIM_SMOOTH,BRAIDING_INDEX
0,0,1,125306.491357,527573.551158,124405.009786,2013-04-02,5881.519719,4706.465915,1.176616
1,1,1,141533.260363,527573.551158,131616.911104,2013-04-28,6181.598208,4798.390830,1.199598
2,2,1,239795.287334,527573.551158,236189.348006,2013-06-15,6601.706987,5081.714263,1.270429
3,3,1,305603.775169,527573.551158,294785.952139,2013-08-02,7081.830913,5554.341860,1.388585
4,4,1,279460.683319,527573.551158,278108.456690,2013-08-18,6481.676044,5089.323290,1.272331
...,...,...,...,...,...,...,...,...,...
1167,1167,4,200129.875614,527573.551158,196523.950792,2011-05-25,6421.660288,4972.174738,1.243044
1168,1168,4,212750.698667,527573.551158,204186.552740,2011-06-10,6421.660484,4922.850387,1.230713
1169,1169,4,265036.907988,527573.551158,261430.947612,2011-08-29,6421.660565,4993.622772,1.248406
1170,1170,4,265036.894202,527573.551158,265938.356048,2011-09-14,6661.722153,5293.677147,1.323419


In [19]:
# NO OVERLAPS... (need a meeting with Barbara)
df_g = df.groupby(['SENSING_TIME', 'ID'])
for key, item in df_g:
    if(item.shape[0] <= 1):
        continue;
    print(item);